## Learning the right layers: a data-driven semi-supervised learning model for multilayer graphs - SYNTHETIC DATASETS

In [1]:
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.neighbors import kneighbors_graph
import matplotlib.pyplot as plt
import math
import sklearn
import scipy
from math import exp,log
from copy import deepcopy
from sklearn.preprocessing import normalize
import seaborn as sns
import pandas as pd
from tqdm.auto import tqdm
import pickle
import numdifftools as nd
import networkx as nx
from matplotlib import pyplot, patches
from scipy import io
import random
import scipy.io
import os
import mat73
from joblib import Parallel, delayed

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Accuracy functions

In [2]:
# INPUT: E vector expected communities
#        P vector predicted communities
def wrong(E,P):
    
    T = sklearn.metrics.confusion_matrix(E,P) #confusion matrix
    
    n=0 #counts number of nodes in the wrong community
    while T.size != 0:
        M = np.max(T)
        [x,y] = np.where(T==M)
        #use x[0] and y[0] because maybe more entries correspond to maximum value 
        n += np.sum(T[x[0],:]) - T[x[0],y[0]]
        n += np.sum(T[:,y[0]]) - T[x[0],y[0]]
        T = np.delete(T, x[0], axis=0)
        T = np.delete(T, y[0], axis=1)
        
    return n

#accuracy
def accuracy(E,P):
    
    n = wrong(E,P) #number of nodes in the wrong community
    C_exp_len = len(E)
    acc = (C_exp_len-wrong(E,P))/C_exp_len
    
    return acc

def accuracy_final(sol_final,C_indexes_known_union,C_exp):
    labels_found = np.argmax(sol_final, axis=1)
    labels_found = labels_found.transpose()
    labels_found = np.delete(labels_found, obj=C_indexes_known_union) #not consider labeled nodes 
    return accuracy(C_exp,labels_found.A1)

def accuracy_Yte(sol_final,labels,index_test):
    labels_found_test = np.argmax(sol_final[index_test,:], axis=1)
    labels_found_test = labels_found_test.transpose()
    return accuracy(labels[index_test],labels_found_test.A1)

In [3]:
def draw_adjacency_matrix(G, node_order=None, partitions=[], colors=[]):
    """
    - G is a netorkx graph
    - node_order (optional) is a list of nodes, where each node in G
          appears exactly once
    - partitions is a list of node lists, where each node in G appears
          in exactly one node list
    - colors is a list of strings indicating what color each
          partition should be
    If partitions is specified, the same number of colors needs to be
    specified.
    """
    adjacency_matrix = nx.to_numpy_matrix(G, dtype=np.bool, nodelist=node_order)

    #Plot adjacency matrix in toned-down black and white
    fig = pyplot.figure(figsize=(5, 5)) # in inches
    pyplot.imshow(adjacency_matrix,
                  cmap="Greys",
                  interpolation="none")
    
    # The rest is just if you have sorted nodes by a partition and want to
    # highlight the module boundaries
    assert len(partitions) == len(colors)
    ax = pyplot.gca()
    for partition, color in zip(partitions, colors):
        current_idx = 0
        for module in partition:
            ax.add_patch(patches.Rectangle((current_idx, current_idx),
                                          len(module), # Width
                                          len(module), # Height
                                          facecolor="none",
                                          edgecolor=color,
                                          linewidth="1"))
            current_idx += len(module)

In [4]:
def unison_shuffled_copies(a, b, c,r):
    assert len(a) == len(b)
    np.random.seed(r)
    p = np.random.permutation(len(a))
    return a[p], b[p], c[b]

### x_sol - solution lower level problem

In [5]:
def x_sol(A,N,C,K,lam,theta,y):
    
    a = theta[K]
    if -1e-2<a<0: 
        a = -1e-2

    I = scipy.sparse.identity(N).tocsc()

    M_a = scipy.sparse.csr_matrix((N,N))
    for k in range(0,K):
        M_k = A[k]
        M_k.eliminate_zeros()
        M_a_k = scipy.sparse.csr_matrix.power(M_k,a)
        M_a_k.data[M_a_k.data == np.inf] = 1e+300
        M_a_k.data[M_a_k.data == -np.inf] = -1e+300
        M_a_k = theta[k]*M_a_k
        M_a += M_a_k
    M_a.eliminate_zeros()
    M_a = scipy.sparse.csr_matrix.power(M_a,1/a)
    M_a.data[M_a.data == np.inf] = 1e+300
    M_a.data[M_a.data == -np.inf] = -1e+300
      
    D_a = scipy.sparse.diags(sum(M_a).toarray()[0]).tocsc()
    
    #solve system using fixed point iterations
    B = lam*scipy.sparse.linalg.inv(I + lam*D_a)
    BM = B.dot(M_a)
    By = B.dot(y)
    x_k = scipy.sparse.csr_matrix((N,C))
    for i in range(0,100): #10 iterations
        x_k = BM.dot(x_k) + By
    sol = x_k
    
    return normalize(sol, norm='l1', axis=0)

### ZOFW - Zeroth order Frank Wolfe

In [6]:
#Frank Wolfe #inexact gradient #Armijo rule 
sign = lambda x: (1, -1)[x<0]

def ZOFW(x_0,fun_theta,K,Y_te,x_sol_theta_dense,tol):
    #tol = 1e-4 #tollerance method
    max_it = 1e3 #max number iterations
    
    box_a = [-20,20] #box alpha variable
    
    #initialization 
    it = 0
    x_it = x_0
        
    fun_x_it_old = math.inf
    fun_x_it = fun_theta(x_it)
             
    #gradient - Finite-Difference Approximations
    c_it = 1e-4
    grad_x_it = np.zeros(K+1)
    for k in range(0,K+1):
        grad_x_it[k] = (fun_theta(x_it+c_it*np.eye(1,K+1,k)[0]) - fun_x_it ) / c_it 
    grad_a = grad_x_it[-1]
    grad_theta = grad_x_it[:-1]   
        
    #minimize linearize function
    #beta #unit simplex #vector of the canonical basis corrispondent to min index gradient 
    x_it_theta = np.eye(1,K,np.argmin(grad_theta))
    #alpha #box #sign
    if grad_a>0:
        x_it_alpha = box_a[0]
    else:
        x_it_alpha = box_a[1]
    #argmin  
    x_it_min = np.append(x_it_theta,x_it_alpha)
        
    #descent direction
    d_it = x_it_min - x_it
    
    while it<max_it and grad_x_it.dot(d_it) <= -tol and fun_x_it_old-fun_x_it>1e-6:

        it += 1
        fun_x_it_old = fun_x_it 
             
        #step size #Armijo rule
        step = 1
        x_it_d = x_it + step*d_it
        fun_x_it_d = fun_theta(x_it_d)       
        while fun_x_it_d > fun_x_it + 0.1*step*(d_it.dot(grad_x_it)):
            step = step/2
            x_it_d = x_it + step*d_it
            fun_x_it_d = fun_theta(x_it_d)  
        else:
            x_it = x_it_d
        
        fun_x_it = fun_theta(x_it)                 
    
        #gradient - Finite-Difference Approximations
        c_it = c_it/2
        grad_x_it = np.zeros(K+1)
        for k in range(0,K+1):
            grad_x_it[k] = (fun_theta(x_it+c_it*np.eye(1,K+1,k)[0]) - fun_x_it ) / c_it 
        grad_a = grad_x_it[-1]
        grad_theta = grad_x_it[:-1] 
        
        #minimize linearize function
        #beta #unit simplex #vector of the canonical basis corrispondent to min index gradient 
        x_it_theta = np.eye(1,K,np.argmin(grad_theta))
        #alpha #box #sign
        if grad_a>0:
            x_it_alpha = box_a[0]
        else:
            x_it_alpha = box_a[1]
        #argmin  
        x_it_min = np.append(x_it_theta,x_it_alpha)
    
        #descent direction
        d_it = x_it_min - x_it 
        
    return x_it,it

### Cross_entropy - loss function upper level problem

In [7]:
def cross_entropy(N,C,x,x_a):
    
    x = np.transpose(x.toarray())
    x_a = x_a.toarray()
    
    x_a_ = np.copy(x_a)
    x_a_[x_a_<=1e-15] = 1e-15 #otherwise log(0)=-inf
    log_x_a = np.log(x_a_)
    x_log_x_a = np.matmul(x,log_x_a).item()
    
    x_a_ = 1 - np.copy(x_a)
    x_a_[x_a_<=1e-15] = 1e-15 #otherwise log(0)=-inf
    log_1_x_a = np.log(x_a_)
    x_log_1_x_a = np.matmul(1-x,log_1_x_a).item()
    
    val = -(1/N) * (x_log_x_a + x_log_1_x_a)
    return val


def cross_entropy_multiclass(N,C,x,x_a):

    num = x_a.toarray()
    den = scipy.sparse.csr_matrix.sum(x_a,axis=1)
    den[den==0] = 1 #can not divide by zero 
    frac = num/den
    frac[frac<=1e-15] = 1e-15 #otherwise log(0)=-inf   
    #log
    frac_log = np.log(frac)
    
    #cross-entropy
    val = -(1/N) * (np.multiply(x.toarray(),frac_log)).sum() 
    
    return val

### Parallelization functions

In [8]:
def cross_entropy_c(A,N,K,C,lam,Y_tr,Y_te,Y,x_0,cross_entropy,c):
    x_sol_theta = lambda theta: x_sol(A,N,1,K,lam,theta,Y_tr[:,c]) 
    fun_theta = lambda theta: cross_entropy(N,C,Y_te[:,c],x_sol_theta(theta))
    theta,it_c = ZOFW(x_0,fun_theta,K,Y_te[:,c],x_sol_theta,1e-4)

    #recalculate solution using selected alpha and all known labels
    x_final = x_sol(A,N,1,K,lam,theta,Y[:,c])
    loss = cross_entropy(N,C,Y[:,c],x_final)
    
    return x_final,theta,loss

def multistart(A,N,K,C,lam,Y_tr,Y_te,Y,cross_entropy,cross_entropy_c,list_x_0,rrrr):
    #initialization FW
    x_0 = list_x_0[:,rrrr]
    
    x_final_c,theta_c,loss_c = zip(*Parallel(n_jobs=2)(delayed(cross_entropy_c)(A,N,K,C,lam,Y_tr,Y_te,Y,x_0,cross_entropy,c) for c in range(C)))
    sol_final = scipy.sparse.hstack(x_final_c)
    theta = np.transpose(np.asarray(theta_c))
    loss = sum(loss_c)
    
    return sol_final,theta,loss

def multistart_multi(A,N,K,C,lam,theta,Y_tr,Y_te,Y,cross_entropy_multi,fun_theta,x_sol_theta,list_x_0,rrrr):
    
    #initialization FW
    x_0 = list_x_0[:,rrrr]
    theta,it = ZOFW(x_0,fun_theta,K,Y_te,x_sol_theta,1e-8)
    
    #recalculate solution using selected alpha and all known labels           
    sol_final = x_sol(A,N,C,K,lam,theta,Y)
    loss = cross_entropy_multi(N,C,Y,sol_final)
    
    return sol_final,theta,loss


def methods(A,N,K,C,lam,Y,labels_known_size,perc_,labels,C_indexes_known,sample2,rrr):
    
    labels_test_size = np.rint(labels_known_size*(perc_/100),out=np.zeros(C,int),casting='unsafe')
    labels_train_size = labels_known_size - labels_test_size
    
    index_train = []
    index_test = []
    for c in range(0,C):   
        shift = int(labels_train_size[c]/(2*sample2))
        index_train_c = C_indexes_known[c][rrr*shift:labels_train_size[c]+rrr*shift] #each sample shift windows 
        index_train.append(index_train_c)
        index_test.append(np.setdiff1d(C_indexes_known[c],index_train_c))
    index_train = np.concatenate(index_train)
    index_test = np.concatenate(index_test)
    
    #matrices Y_tr and Y_te #corrispondent to labels training and validation sets
    row = index_train
    comm_list_train = []
    for c in range(0,C):
        comm_list_train.append([c]*labels_train_size[c])
    comm_list_train = np.concatenate(comm_list_train)
    col = np.array(comm_list_train)
    data = np.array(np.ones(sum(labels_train_size)))
    Y_tr = scipy.sparse.csr_matrix((data, (row, col)), shape=(N, C))
    # normalized
    Y_tr = normalize(Y_tr, norm='l1', axis=0)

    row = index_test
    comm_list_test = []
    for c in range(0,C):
        comm_list_test.append([c]*labels_test_size[c])
    comm_list_test = np.concatenate(comm_list_test)
    col = np.array(comm_list_test)
    data = np.array(np.ones(sum(labels_test_size)))
    Y_te = scipy.sparse.csr_matrix((data, (row, col)), shape=(N, C))
    # normalized
    Y_te = normalize(Y_te, norm='l1', axis=0)

    #multi-start #list random initial points FW
    random.seed(rrr)
    np.random.seed(rrr)
    multistart_n = 10
    list_x_0 = np.zeros((K+1,multistart_n))
    for rrrr in range(0,multistart_n):
        list_x_0[:,rrrr] = np.append(np.random.dirichlet(np.ones(K),size=1),random.uniform(-20,20))
    #last two initial points fixed [1/K,..,1/k,1], [1/K,..,1/k,-1]
    u = np.ones(K+1) / K #theta sum 1 
    u[K] = 1 #alpha
    w = u.copy()
    w[K] = -1
    list_x_0[:,0] = u
    list_x_0[:,1] = w


    #loss function: binomial classification, each class separate
    #save value function each point #select minimum and corrispondent solution and variables 
    sol_final_multistart,theta_multistart,loss_multistart = zip(*Parallel(n_jobs=2,prefer="threads")(delayed(multistart)(A,N,K,C,lam,Y_tr,Y_te,Y,cross_entropy,cross_entropy_c,list_x_0,rrrr) for rrrr in range(multistart_n)))

    #choose sol corrispondent to min vaue loss 
    index_min = loss_multistart.index(min(loss_multistart))
    sol_final = sol_final_multistart[index_min]
    theta = theta_multistart[index_min]
 
    #calculate accuracy just over Y_tr, in order to choose better sample Y_tr from Y 
    acc_k = accuracy_Yte(sol_final,labels,index_test)
    sol_final_0 = sol_final
    matrix_acc_0 = acc_k
    matrix_theta_0 = theta

    #loss function: multiclass cross-entropy
    x_sol_theta = lambda theta: x_sol(A,N,C,K,lam,theta,Y_tr)
    fun_theta = lambda theta: cross_entropy_multiclass(N,C,Y_te,x_sol_theta(theta))
    sol_final_multistart,theta_multistart,loss_multistart = zip(*Parallel(n_jobs=2,prefer="threads")(delayed(multistart_multi)(A,N,K,C,lam,theta,Y_tr,Y_te,Y,cross_entropy_multiclass,fun_theta,x_sol_theta,list_x_0,rrrr) for rrrr in range(multistart_n)))

    #choose sol corrispondent to min vaue loss 
    index_min = loss_multistart.index(min(loss_multistart))
    sol_final = sol_final_multistart[index_min]
    theta = theta_multistart[index_min]

    #calculate accuracy just over Y_tr, in order to choose better sample Y_tr from Y 
    acc_k = accuracy_Yte(sol_final,labels,index_test)
    sol_final_1 = sol_final
    matrix_acc_1 = acc_k
    matrix_theta_1 = np.array([theta]*C).transpose()
    
    return matrix_acc_0,sol_final_0,matrix_theta_0,matrix_acc_1,sol_final_1,matrix_theta_1

### Datasets

#### synthetic_datasets

In [3]:
def synthetic_datasets(N,C,K,case,cluster_std,knn,perc,lam):
    
    #random samples
    sample1 = 5 #5 #random matrices A
    sample2 = 5 #5 #division training and test sets Y_tr Y_te #just for proposed methods 

    #list accuracy methods #for each random marix A 
    matrix_acc = np.zeros((K+5+2,sample1))  
    #list values theta and alpha porposed methods 
    matrix_theta = np.zeros((sample1,2,K+1,C))

    #save for state-of the-art-methods in Matlab
    A_save = []
    labels_save = []
    Y_save = []
    for r in tqdm(range(0,sample1)): #random matrices

        #generate weigthed random networks 
        n = 5 #space dimention
        random_state = r #random sample
        X, labels, centers = make_blobs(n_samples=N, centers=C, n_features=n, random_state=random_state,cluster_std=cluster_std, return_centers=True)
        #plt.scatter(X[:, 0], X[:, 1], c=labels)

        #rename labels from 0 
        labels_unique,labels = np.unique(labels, return_inverse=True)  
        # communties size
        C_size = np.zeros(C)
        for com in range(0,C):
            com_ind = np.where(labels == com)
            C_size[com] = len(com_ind[0])

        #generate correspondent affinity matrices
        A = [] #list #A[k] sparse adjacency matrix layer k

        if case=='info2': #informative case #different nodes
            if K>1:
                #1 layer: Euclidean distance
                W = kneighbors_graph(X, n_neighbors=knn, mode='distance', include_self=False)
                W = 0.5 * (W + W.T)
                index_com = np.where(labels == 0)
                for com in range(1,C): 
                    index_com = np.concatenate((index_com, np.where(labels == com)), axis=None)
                W = W[index_com, :]
                W = W[:, index_com]
                A.append(W)
                #others layers random #new generated network #Euclidean distance
                for k in range(1,K):
                    X_, labels_, centers_ = make_blobs(n_samples=N, centers=C, n_features=n, random_state=r+random_state+k,cluster_std=cluster_std, return_centers=True)
                    W_ = kneighbors_graph(X_, n_neighbors=knn, mode='distance', include_self=False)
                    W_ = 0.5 * (W_ + W_.T)
                    #rename labels from 0 
                    labels_unique_,labels_ = np.unique(labels_, return_inverse=True)  
                    # communties size
                    index_com_ = np.where(labels_ == 0)
                    for com in range(1,C): 
                        index_com_ = np.concatenate((index_com_, np.where(labels_ == com)), axis=None)
                    W_ = W_[index_com_, :]
                    W_ = W_[:, index_com_]                   
                    A.append(W_)


        if case=='noisy': #noisy case 
            if K>1:
                #1 layer: Euclidean distance
                W = kneighbors_graph(X, n_neighbors=knn, mode='distance', include_self=False)
                W = 0.5 * (W + W.T)
                A.append(W)
                #2 layer: random 
                #others layers random 
                for k in range(1,K):
                    X_, labels_, centers_ = make_blobs(n_samples=N, centers=C, n_features=n, random_state=r+random_state+k,cluster_std=cluster_std, return_centers=True)
                    W_ = kneighbors_graph(X_, n_neighbors=knn, mode='distance', include_self=False)
                    W_ = 0.5 * (W_ + W_.T)
                    index = np.arange(N)
                    np.random.shuffle(index)
                    W_ = W_[index, :]
                    W_ = W_[:, index]
                    A.append(W_)

        if case=='compl': #complementary case #each layer one community 
            W_list = []
            #1 layer eucledian #info 
            W = kneighbors_graph(X, n_neighbors=knn, mode='distance', include_self=False)
            W = 0.5 * (W + W.T)
            A.append(W)
            #others layers random #less connections
            knn_ = int(knn/3)    
            for k in range(1,K+1):
                X_, labels_, centers_ = make_blobs(n_samples=N, centers=C, n_features=n, random_state=r+random_state+k,cluster_std=cluster_std, return_centers=True)
                W_ = kneighbors_graph(X_, n_neighbors=knn_, mode='distance', include_self=False)
                W_ = 0.5 * (W_ + W_.T)
                index = np.arange(N)
                np.random.shuffle(index)
                W_ = W_[index, :]
                W_ = W_[:, index]
                A.append(W_)

        if case!='info2':
            #reorder A adjacency matrix by communities
            index_com = np.where(labels == 0)
            for com in range(1,C): 
                index_com = np.concatenate((index_com, np.where(labels == com)), axis=None)
            for k in range(0,K):
                A[k] = A[k][index_com, :]
                A[k] = A[k][:, index_com]
                
        labels = []
        for com in range(0,C):
            labels = np.concatenate((labels, [com]*int(C_size[com])), axis=None)

        # communities indeces 
        C_indexes = np.zeros(C, dtype=object)
        for com in range(0,C):
            com_ind = np.where(labels == com)
            C_indexes[com] = com_ind

        # for each community a percentage per of labels are known
        labels_known_size = np.rint(C_size*(perc/100),out=np.zeros(C,int),casting='unsafe') #number of known labels in each community
        labels_known_size[np.where(labels_known_size < 2)] = 2 #at least 2 node labeled each community
        labels_known_size[np.where(C_size == 1)] = 1 #unless community just one node

        if case=='compl':
            #one community each layer
            A_copy = []
            for k in range(0,K):
                A_copy.append(A[k].copy())
            sum_com = 0
            for k in range(0,K):
                sum_com_old = sum_com
                sum_com += int(C_size[k])
                A[k+1][sum_com_old:sum_com, sum_com_old:sum_com] = A_copy[0][sum_com_old:sum_com, sum_com_old:sum_com]
            del A[0]

        #normalize matrices 
        for k in range(0,K):
            A[k] = A[k]/A[k].max()

        A_save.append(A)
        labels_save.append(labels)

#         #plot communities
#         for k in range(0,K):
#             G = nx.from_scipy_sparse_matrix(A[k])
#             draw_adjacency_matrix(G) 

        #Y known labels
        np.random.seed(0) 
        C_indexes_known = np.zeros(C, dtype=object)
        for com in range(0,C):
            C_indexes_known[com] = np.random.choice(C_indexes[com][0],labels_known_size[com],replace=False)

        # Y matrix #known labels
        C_indexes_known_union = np.concatenate(C_indexes_known)
        comm_list = []
        for c in range(0,C):
            comm_list.append([c]*labels_known_size[c])
        comm_list = np.concatenate(comm_list)
        row = C_indexes_known_union
        col = np.array(comm_list)
        data = np.array(np.ones(sum(labels_known_size)))
        Y = scipy.sparse.csr_matrix((data, (row, col)), shape=(N, C))
        # normalized
        Y = normalize(Y, norm='l2', axis=0) #check scipy.sparse.linalg.norm(Y, axis=0)

        # expected communities (without known labels) #to calculate final accuracy
        C_exp = np.delete(labels, obj=C_indexes_known_union)

        Y_save.append(Y)

        #acc_single_layers
        for k in range(0,K):
            theta = np.ones((1,2))[0]
            sol_final = x_sol([A[k]],N,C,1,lam,theta,Y)
            acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
            matrix_acc[k,r] = acc_k
            
        #alpha=1 #arithmetic mean     
        a = 1 #alpha 1
        theta = np.ones(K+1) / K #theta sum 1 
        theta[-1] = 1 #alpha 1
        sol_final = x_sol(A,N,C,K,lam,theta,Y)
        acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
        matrix_acc[K,r]  = acc_k

        #alpha=-1 #harmonic mean    
        sol_final = scipy.sparse.lil_matrix((N,C))
        theta = np.ones(K+1) / K #theta sum 1 
        theta[-1] = -1 #alpha -1
        sol_final = x_sol(A,N,C,K,lam,theta,Y)
        acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
        matrix_acc[K+1,r]  = acc_k

        #alpha=+inf #maximum    
        A_max = A[0]
        for k in range(0,K-1):
            A_max = A_max.maximum(A[k+1])
        theta = np.ones((1,2))[0]
        sol_final = x_sol([A_max],N,C,1,lam,theta,Y)
        acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
        matrix_acc[K+2,r]  = acc_k

        #alpha=-inf #minimum    
        A_min = A[0]
        for k in range(0,K-1):
            A_min = A_min.minimum(A[k+1])
        theta = np.ones((1,2))[0]
        sol_final = x_sol([A_min],N,C,1,lam,theta,Y)
        acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
        matrix_acc[K+3,r]  = acc_k

        #alpha=0 #geometric mean    
        A_prod = A[0]
        for k in range(0,K-1):
            A_prod = A_prod.multiply(A[k+1])
        A_prod = scipy.sparse.csr_matrix.power(A_prod,1/K)
        theta = np.ones((1,2))[0]
        sol_final = x_sol([A_prod],N,C,1,lam,theta,Y)
        acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
        matrix_acc[K+4,r]  = acc_k 

        #proposed methods #parallelize sampling training and test sets
        perc_ = 80 #percentage test set
        matrix_acc_0,sol_final_0,matrix_theta_0,matrix_acc_1,sol_final_1,matrix_theta_1 = zip(*Parallel(n_jobs=2,prefer="threads")(delayed(methods)(A,N,K,C,lam,Y,labels_known_size,perc_,labels,C_indexes_known,sample2,rrr) for rrr in range(sample2)))

        argmax0 = np.argmax(matrix_acc_0)
        sol_final = sol_final_0[argmax0]
        acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
        matrix_acc[K+5,r]  = acc_k
        matrix_theta[r,0,:,:]=matrix_theta_0[argmax0]

        argmax1 = np.argmax(matrix_acc_1)
        sol_final = sol_final_1[argmax1]
        acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
        matrix_acc[K+6,r]  = acc_k
        matrix_theta[r,1,:,:]=matrix_theta_1[argmax1]
                
                
    #save matrix results samples
    with open('Results/matrix_acc_theta_N'+str(N)+'_C'+str(C)+'_case '+case+'_std'+str(cluster_std)+'_knn'+str(knn)+'_perc'+str(perc)+'_lam'+str(lam)+'_sample'+str(sample1),"wb") as fp:
        pickle.dump(matrix_acc,fp)
    with open('Results/matrix_theta_theta_N'+str(N)+'_C'+str(C)+'_case '+case+'_std'+str(cluster_std)+'_knn'+str(knn)+'_perc'+str(perc)+'_lam'+str(lam)+'_sample'+str(sample1),"wb") as fp:
        pickle.dump(matrix_theta,fp)
        
    #save for state-of the-art-methods in Matlab
    my_file = 'Results/Datasets_Matlab/Matlab_theta_N'+str(N)+'_C'+str(C)+'_case '+case+'_std'+str(cluster_std)+'_knn'+str(knn)+'_perc'+str(perc)+'_lam'+str(lam)+'_sample'+str(sample1)+".mat"
    scipy.io.savemat(my_file, dict(A_list=A_save, Y_list=Y_save, labels_list=labels_save))
    
    return A_save,labels_save,Y_save,matrix_acc,matrix_theta

#### real_datasets

In [ ]:
def info_datasets(datasets_list):
    for dataset in datasets_list: 
        my_path = 'Datasets'   
        my_file = dataset+".mat"
        file_path = os.path.join(my_path, my_file)
        data = scipy.io.loadmat(file_path)
        X = data['W_cell']
        if dataset in ["UCI_mfeat","cora","citeseer"]:
            labels = data['labels'].toarray().transpose()[0]
        else:
            labels = data['labels'].toarray()[0]
        #information 
        N = len(labels) #number of nodes
        #rename labels from 0 
        labels_unique,labels = np.unique(labels, return_inverse=True) 
        C = len(labels_unique) #number of communities 
        K = len(X) #number layers
        print(dataset + ' - nodes: ' + str(N) + ', communities: ' + str(C) + ', layers: ' + str(K))

In [ ]:
def real_datasets(dataset,knn,perc,lam): 
    
    #random samples
    sample1 = 5 #known labels Y
    sample2 = 5 #division training and test sets Y_tr Y_te #just for proposed methods 
    
    #download matrix 
    my_path = 'Datasets'   
    my_file = dataset+".mat"
    file_path = os.path.join(my_path, my_file)
    data = scipy.io.loadmat(file_path)
    X = data['W_cell']
    if dataset in ["UCI_mfeat","cora","citeseer"]:
        labels = data['labels'].toarray().transpose()[0]
    else:
        labels = data['labels'].toarray()[0]
    #information 
    N = len(labels) #number of nodes
    #rename labels from 0 
    labels_unique,labels = np.unique(labels, return_inverse=True) 
    C = len(labels_unique) #number of communities 
    K = len(X) #number layers
    
    #list accuracy methods #for each random marix A 
    matrix_acc = np.zeros((K+5+2,sample1))  
    #list values theta and alpha porposed methods 
    matrix_theta = np.zeros((sample1,2,K+1,C))
    
    # communities indeces # communties size
    C_size = np.zeros(C)
    for com in range(0,C):
        com_ind = np.where(labels == com)
        #C_indexes[com] = com_ind
        C_size[com] = len(com_ind[0])

    #reorder A adjacency matrix by communities
    index_com = np.where(labels == 0)
    for com in range(1,C): 
        index_com = np.concatenate((index_com, np.where(labels == com)), axis=None)

    A = list(X)
    for k in range(0,K):
        A[k] = A[k][0]

    for k in range(0,K):
        A[k] = A[k][index_com, :]
        A[k] = A[k][:, index_com]

    labels = []
    for com in range(0,C):
        labels = np.concatenate((labels, [com]*int(C_size[com])), axis=None)

    # communities indeces # communties size
    C_indexes = np.zeros(C, dtype=object)
    for com in range(0,C):
        com_ind = np.where(labels == com)
        C_indexes[com] = com_ind

    # for each community a percentage of labels are known
    #perc = 10 #percentage known labels each community 
    labels_known_size = np.rint(C_size*(perc/100),out=np.zeros(C,int),casting='unsafe') #number of known labels in each community
    labels_known_size[np.where(labels_known_size < 2)] = 2 #at least 2 node labeled each community
    labels_known_size[np.where(C_size == 1)] = 1 #unless community just one node

    # #normalize matrices 
    # for k in range(0,K):
    #     A[k] = A[k]/A[k].max()

    # for k in range(0,K):
    #     G = nx.from_scipy_sparse_matrix(A[k])
    #     draw_adjacency_matrix(G)

    Y_save = [] #save for state-of the-art-methods in Matlab
    for r in tqdm(range(0,sample1)): #random indices know labels Y
        
        # indices know labels each community 
        np.random.seed(r) 
        C_indexes_known = np.zeros(C, dtype=object)
        for com in range(0,C):
            C_indexes_known[com] = np.random.choice(C_indexes[com][0],labels_known_size[com],replace=False)
  
        # Y matrix #known labels
        C_indexes_known_union = np.concatenate(C_indexes_known)
        comm_list = []
        for c in range(0,C):
            comm_list.append([c]*labels_known_size[c])
        comm_list = np.concatenate(comm_list)
        row = C_indexes_known_union
        col = np.array(comm_list)
        data = np.array(np.ones(sum(labels_known_size)))
        Y = scipy.sparse.csr_matrix((data, (row, col)), shape=(N, C))
        # normalized
        Y = normalize(Y, norm='l2', axis=0) #check scipy.sparse.linalg.norm(Y, axis=0)
        
        # expected communities (without known labels) #to calculate final accuracy
        C_exp = np.delete(labels, obj=C_indexes_known_union)

        Y_save.append(Y)
        
        #acc_single_layers
        for k in range(0,K):
            theta = np.ones((1,2))[0]
            sol_final = x_sol([A[k]],N,C,1,lam,theta,Y)
            acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
            matrix_acc[k,r] = acc_k
            
        #alpha=1 #arithmetic mean     
        a = 1 #alpha 1
        theta = np.ones(K+1) / K #theta sum 1 
        theta[-1] = 1 #alpha 1
        sol_final = x_sol(A,N,C,K,lam,theta,Y)
        acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
        matrix_acc[K,r]  = acc_k

        #alpha=-1 #harmonic mean    
        sol_final = scipy.sparse.lil_matrix((N,C))
        theta = np.ones(K+1) / K #theta sum 1 
        theta[-1] = -1 #alpha -1
        sol_final = x_sol(A,N,C,K,lam,theta,Y)
        acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
        matrix_acc[K+1,r]  = acc_k

        #alpha=+inf #maximum    
        A_max = A[0]
        for k in range(0,K-1):
            A_max = A_max.maximum(A[k+1])
        theta = np.ones((1,2))[0]
        sol_final = x_sol([A_max],N,C,1,lam,theta,Y)
        acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
        matrix_acc[K+2,r]  = acc_k

        #alpha=-inf #minimum    
        A_min = A[0]
        for k in range(0,K-1):
            A_min = A_min.minimum(A[k+1])
        theta = np.ones((1,2))[0]
        sol_final = x_sol([A_min],N,C,1,lam,theta,Y)
        acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
        matrix_acc[K+3,r]  = acc_k

        #alpha=0 #geometric mean    
        A_prod = A[0]
        for k in range(0,K-1):
            A_prod = A_prod.multiply(A[k+1])
        A_prod = scipy.sparse.csr_matrix.power(A_prod,1/K)
        theta = np.ones((1,2))[0]
        sol_final = x_sol([A_prod],N,C,1,lam,theta,Y)
        acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
        matrix_acc[K+4,r]  = acc_k 

        #proposed methods #parallelize sampling training and test sets
        perc_ = 80 #perc_ = 50 #percentage test set
        matrix_acc_0,sol_final_0,matrix_theta_0,matrix_acc_1,sol_final_1,matrix_theta_1 = zip(*Parallel(n_jobs=2,prefer="threads")(delayed(methods)(A,N,K,C,lam,Y,labels_known_size,perc_,labels,C_indexes_known,sample2,rrr) for rrr in range(sample2)))

        argmax0 = np.argmax(matrix_acc_0)
        sol_final = sol_final_0[argmax0]
        acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
        matrix_acc[K+5,r]  = acc_k
        matrix_theta[r,0,:,:]=matrix_theta_0[argmax0]

        argmax1 = np.argmax(matrix_acc_1)
        sol_final = sol_final_1[argmax1]
        acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
        matrix_acc[K+6,r]  = acc_k
        matrix_theta[r,1,:,:]=matrix_theta_1[argmax1]
                
    #save matrix results samples
    with open("Results_Real/Matlab_acc_theta_"+dataset+'_knn'+str(knn)+'_perc'+str(perc)+'_lam'+str(lam),"wb") as fp:
        pickle.dump(matrix_acc,fp)
    with open("Results_Real/Matlab_theta_theta_"+dataset+'_knn'+str(knn)+'_perc'+str(perc)+'_lam'+str(lam),"wb") as fp:
        pickle.dump(matrix_theta,fp)
    #save for state-of the-art-methods in Matlab 
    my_file = "Results_Real/Datasets_Matlab_Real/Matlab_"+dataset+'_knn'+str(knn)+'_perc'+str(perc)+'_lam'+str(lam)+".mat"
    scipy.io.savemat(my_file, dict(A_list=A, Y_list=Y_save, labels_list=labels))

    return A,labels,Y_save,matrix_acc,matrix_theta

In [ ]:
def real_datasets_noisy(dataset,knn,perc,lam,rand_layers): 
    
    #random samples
    sample1 = 1 #known labels Y
    sample2 = 5 #division training and test sets Y_tr Y_te #just for proposed methods
    sample3 = 3 #random layers added 
       
    #download matrix 
    my_path = 'Datasets'   
    my_file = dataset+".mat"
    file_path = os.path.join(my_path, my_file)
    data = scipy.io.loadmat(file_path)
    X = data['W_cell']
    if dataset in ["UCI_mfeat","cora","citeseer"]:
        labels = data['labels'].toarray().transpose()[0]
    else:
        labels = data['labels'].toarray()[0]
    #information 
    N = len(labels) #number of nodes
    #rename labels from 0 
    labels_unique,labels = np.unique(labels, return_inverse=True) 
    C = len(labels_unique) #number of communities 
    K = len(X) #number layers

    # communities indeces # communties size
    #C_indexes = np.zeros(C, dtype=object)
    C_size = np.zeros(C)
    for com in range(0,C):
        com_ind = np.where(labels == com)
        #C_indexes[com] = com_ind
        C_size[com] = len(com_ind[0])

    #reorder A adjacency matrix by communities
    index_com = np.where(labels == 0)
    for com in range(1,C): 
        index_com = np.concatenate((index_com, np.where(labels == com)), axis=None)

    A_ = list(X)
    for k in range(0,K):
        A_[k] = A_[k][0]

    for k in range(0,K):
        A_[k] = A_[k][index_com, :]
        A_[k] = A_[k][:, index_com]

    labels = []
    for com in range(0,C):
        labels = np.concatenate((labels, [com]*int(C_size[com])), axis=None)

    # communities indeces # communties size
    C_indexes = np.zeros(C, dtype=object)
    for com in range(0,C):
        com_ind = np.where(labels == com)
        C_indexes[com] = com_ind

    # for each community a percentage of labels are known
    #perc = 10 #percentage known labels each community 
    labels_known_size = np.rint(C_size*(perc/100),out=np.zeros(C,int),casting='unsafe') #number of known labels in each community
    labels_known_size[np.where(labels_known_size < 2)] = 2 #at least 2 node labeled each community
    labels_known_size[np.where(C_size == 1)] = 1 #unless community just one node

    # for k in range(0,K):
    #     G = nx.from_scipy_sparse_matrix(A_[k])
    #     draw_adjacency_matrix(G)

    list_acc = [] #save sample3 random layers added 
    list_theta = []
    list_A_save = []
    list_Y_save = []

    K = K + rand_layers #number layers
    #list accuracy methods #for each random marix A 
    matrix_acc = np.zeros((K+5+2,sample1))  
    #list values theta and alpha porposed methods 
    matrix_theta = np.zeros((sample1,2,K+1,C))
    
    for rrr in tqdm(range(0,sample3)): #random layers added 
        A = A_.copy()
        
        #generate weigthed random networks 
        n = 5 #space dimention
        random_state = rrr #random sample
        cluster_std = 8
        for k in range(K-rand_layers,K):  
            X_, labels_, centers_ = make_blobs(n_samples=N, centers=C, n_features=n, random_state=random_state+k,cluster_std=cluster_std, return_centers=True)
            W_ = kneighbors_graph(X_, n_neighbors=knn, mode='distance', include_self=False)
            W_ = 0.5 * (W_ + W_.T)
            index = np.arange(N)
            np.random.shuffle(index)
            W_ = W_[index, :]
            W_ = W_[:, index]
            A.append(W_)
           
        #correspondent adjacency matrix - unweighted 
        for k in range(0,K):
            A[k].data[:] = 1

        # for k in range(0,K):
        #     G = nx.from_scipy_sparse_matrix(A[k])
        #     draw_adjacency_matrix(G)

        Y_save = [] #save for state-of the-art-methods in Matlab
        for r in tqdm(range(0,sample1)): #random indices know labels Y

            # indices know labels each community 
            np.random.seed(r) 
            C_indexes_known = np.zeros(C, dtype=object)
            for com in range(0,C):
                C_indexes_known[com] = np.random.choice(C_indexes[com][0],labels_known_size[com],replace=False)

            # Y matrix #known labels
            C_indexes_known_union = np.concatenate(C_indexes_known)
            comm_list = []
            for c in range(0,C):
                comm_list.append([c]*labels_known_size[c])
            comm_list = np.concatenate(comm_list)
            row = C_indexes_known_union
            col = np.array(comm_list)
            data = np.array(np.ones(sum(labels_known_size)))
            Y = scipy.sparse.csr_matrix((data, (row, col)), shape=(N, C))
            # normalized
            Y = normalize(Y, norm='l2', axis=0) #check scipy.sparse.linalg.norm(Y, axis=0)

            # expected communities (without known labels) #to calculate final accuracy
            C_exp = np.delete(labels, obj=C_indexes_known_union)

            Y_save.append(Y)

            #acc_single_layers
            for k in range(0,K):
                theta = np.ones((1,2))[0]
                sol_final = x_sol([A[k]],N,C,1,lam,theta,Y)
                acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
                matrix_acc[k,r] = acc_k

            #alpha=1 #arithmetic mean     
            a = 1 #alpha 1
            theta = np.ones(K+1) / K #theta sum 1 
            theta[-1] = 1 #alpha 1
            sol_final = x_sol(A,N,C,K,lam,theta,Y)
            acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
            matrix_acc[K,r]  = acc_k

            #alpha=-1 #harmonic mean    
            sol_final = scipy.sparse.lil_matrix((N,C))
            theta = np.ones(K+1) / K #theta sum 1 
            theta[-1] = -1 #alpha -1
            sol_final = x_sol(A,N,C,K,lam,theta,Y)
            acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
            matrix_acc[K+1,r]  = acc_k

            #alpha=+inf #maximum    
            A_max = A[0]
            for k in range(0,K-1):
                A_max = A_max.maximum(A[k+1])
            theta = np.ones((1,2))[0]
            sol_final = x_sol([A_max],N,C,1,lam,theta,Y)
            acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
            matrix_acc[K+2,r]  = acc_k

            #alpha=-inf #minimum    
            A_min = A[0]
            for k in range(0,K-1):
                A_min = A_min.minimum(A[k+1])
            theta = np.ones((1,2))[0]
            sol_final = x_sol([A_min],N,C,1,lam,theta,Y)
            acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
            matrix_acc[K+3,r]  = acc_k

            #alpha=0 #geometric mean    
            A_prod = A[0]
            for k in range(0,K-1):
                A_prod = A_prod.multiply(A[k+1])
            A_prod = scipy.sparse.csr_matrix.power(A_prod,1/K)
            theta = np.ones((1,2))[0]
            sol_final = x_sol([A_prod],N,C,1,lam,theta,Y)
            acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
            matrix_acc[K+4,r]  = acc_k 

            #proposed methods #parallelize sampling training and test sets
            perc_ = 80 #perc_ = 50 #percentage test set
            matrix_acc_0,sol_final_0,matrix_theta_0,matrix_acc_1,sol_final_1,matrix_theta_1 = zip(*Parallel(n_jobs=2,prefer="threads")(delayed(methods)(A,N,K,C,lam,Y,labels_known_size,perc_,labels,C_indexes_known,sample2,rrr) for rrr in range(sample2)))

            argmax0 = np.argmax(matrix_acc_0)
            sol_final = sol_final_0[argmax0]
            acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
            matrix_acc[K+5,r]  = acc_k
            matrix_theta[r,0,:,:]=matrix_theta_0[argmax0]

            argmax1 = np.argmax(matrix_acc_1)
            sol_final = sol_final_1[argmax1]
            acc_k = accuracy_final(sol_final,C_indexes_known_union,C_exp)
            matrix_acc[K+6,r]  = acc_k
            matrix_theta[r,1,:,:]=matrix_theta_1[argmax1]
                
            #save in list 
            list_acc.append(matrix_acc)  
            list_theta.append(matrix_theta)
            list_A_save.append(A)
            list_Y_save.append(Y_save)

            #save matrix results samples 
            with open("Results_Real/Matlab_acc_theta_"+dataset+'_knn'+str(knn)+'_perc'+str(perc)+'_lam'+str(lam)+'_Knoisy'+str(rand_layers)+'_rrr'+str(r),"wb") as fp:
                pickle.dump(list_acc,fp)
            with open("Results_Real/Matlab_theta_theta_"+dataset+'_knn'+str(knn)+'_perc'+str(perc)+'_lam'+str(lam)+'_Knoisy'+str(rand_layers)+'_rrr'+str(r),"wb") as fp:
                pickle.dump(list_theta,fp)
            #save for state-of the-art-methods in Matlab 
            my_file = "Results_Real/Datasets_Matlab_Real/Matlab_"+dataset+'_knn'+str(knn)+'_perc'+str(perc)+'_lam'+str(lam)+'_Knoisy'+str(rand_layers)+'_rrr'+str(r)+".mat"
            scipy.io.savemat(my_file, dict(A_list=list_A_save, Y_list=list_Y_save, labels_list=labels))
                
    #save matrix results samples 
    with open("Results_Real/Matlab_acc_theta_"+dataset+'_knn'+str(knn)+'_perc'+str(perc)+'_lam'+str(lam)+'_Knoisy'+str(rand_layers),"wb") as fp:
        pickle.dump(list_acc,fp)
    with open("Results_Real/Matlab_theta_theta_"+dataset+'_knn'+str(knn)+'_perc'+str(perc)+'_lam'+str(lam)+'_Knoisy'+str(rand_layers),"wb") as fp:
        pickle.dump(list_theta,fp)
    #save for state-of the-art-methods in Matlab 
    my_file = "Results_Real/Datasets_Matlab_Real/Matlab_"+dataset+'_knn'+str(knn)+'_perc'+str(perc)+'_lam'+str(lam)+'_Knoisy'+str(rand_layers)+".mat"
    scipy.io.savemat(my_file, dict(A_list=list_A_save, Y_list=list_Y_save, labels_list=labels))

    return list_A_save,labels,list_Y_save,list_acc,list_theta

### results_statistics

In [4]:
def results_statistics_synthetic(N,C,K,case,cluster_std,knn,perc,lam):
    
    sample1 = 5
    
    #download matrix
    with open('Results/matrix_acc_theta_N'+str(N)+'_C'+str(C)+'_case '+case+'_std'+str(cluster_std)+'_knn'+str(knn)+'_perc'+str(perc)+'_lam'+str(lam)+'_sample'+str(sample1),"rb") as fp:
        matrix_acc = pickle.load(fp)
    with open('Results/matrix_theta_theta_N'+str(N)+'_C'+str(C)+'_case '+case+'_std'+str(cluster_std)+'_knn'+str(knn)+'_perc'+str(perc)+'_lam'+str(lam)+'_sample'+str(sample1),"rb") as fp:
        matrix_theta = pickle.load(fp)
        
    K = len(matrix_acc)-(5+2) #layers  
    
    method_names = []
    for i in range(0,K):
        method_names.append('layer '+str(i+1))
    method_names = method_names + ['arithmetic', 
    'harmonic',
    'maximum',
    'minimum',
    'geometric',
    'binomial',
    'multi']

    #mean and std all methods respect to random networks
    matrix_acc_mean = np.mean(matrix_acc,axis=1)
    matrix_acc_std = np.std(matrix_acc,axis=1)
    
    matrix_acc_mean_df = pd.DataFrame(list(zip(method_names,matrix_acc_mean,matrix_acc_std)))
    matrix_acc_mean_df.columns = ['method', 'acc_mean','acc_std']
    #matrix_acc_mean_df = matrix_acc_mean_df.sort_values(by=['acc_mean'], ascending=False)
    

    #mean and std all methods respect to random networks
    matrix_theta_mean = np.mean(matrix_theta,axis=0)
    matrix_theta_std = np.std(matrix_theta,axis=0)

    df1 = pd.DataFrame(list(matrix_theta_mean[0]))
    df1.insert(0, 'method/community', method_names[-2])
    df2 = pd.DataFrame(list(matrix_theta_mean[1]))
    df2.insert(0, 'method/community', method_names[-1])

    matrix_theta_mean_df = pd.concat([df1, df2], axis = 0)
    
    return matrix_acc_mean_df,matrix_theta_mean_df

In [5]:
def results_statistics_real(dataset,knn,perc,lam):
    
    sample1 = 5
    
    #download matrix
    with open("Results_Real/Matlab_acc_theta_"+dataset+'_knn'+str(knn)+'_perc'+str(perc)+'_lam'+str(lam),"rb") as fp:
        matrix_acc = pickle.load(fp)
    with open("Results_Real/Matlab_theta_theta_"+dataset+'_knn'+str(knn)+'_perc'+str(perc)+'_lam'+str(lam),"rb") as fp:
        matrix_theta = pickle.load(fp)
        
    K = len(matrix_acc)-(5+2) #layers  
    
    method_names = []
    for i in range(0,K):
        method_names.append('layer '+str(i+1))
    method_names = method_names + ['arithmetic', 
    'harmonic',
    'maximum',
    'minimum',
    'geometric',
    'binomial',
    'multi']

    #mean and std all methods respect to random networks
    matrix_acc_mean = np.mean(matrix_acc,axis=1)
    matrix_acc_std = np.std(matrix_acc,axis=1)
    
    matrix_acc_mean_df = pd.DataFrame(list(zip(method_names,matrix_acc_mean,matrix_acc_std)))
    matrix_acc_mean_df.columns = ['method', 'acc_mean','acc_std']
    #matrix_acc_mean_df = matrix_acc_mean_df.sort_values(by=['acc_mean'], ascending=False)
    

    #mean and std all methods respect to random networks
    matrix_theta_mean = np.mean(matrix_theta,axis=0)
    matrix_theta_std = np.std(matrix_theta,axis=0)
    
    df1 = pd.DataFrame(list(matrix_theta_mean[1]))
    df1.insert(0, 'method/community', method_names[-2])
    df2 = pd.DataFrame(list(matrix_theta_mean[2]))
    df2.insert(0, 'method/community', method_names[-1])

    matrix_theta_mean_df = pd.concat([df1, df2], axis = 0)
    
    return matrix_acc_mean_df,matrix_theta_mean_df

In [44]:
#dict number communities
datasets_list = ["3sources","BBCSport2view_544","BBC4view_685","WikipediaArticles","UCI_mfeat","cora","citeseer","dkpol","aucs"]
dict_dataset_C = dict(zip(datasets_list,[6,5,5,10,10,7,6,10]))

def results_statistics_real_noisy(dataset,knn,perc,lam,rand_layers):
    
    #download matrix
    with open("Results_Real/Matlab_acc_theta_"+dataset+'_knn'+str(knn)+'_perc'+str(perc)+'_lam'+str(lam)+'_Knoisy'+str(rand_layers),"rb") as fp:
        list_acc = pickle.load(fp)
    with open("Results_Real/Matlab_theta_theta_"+dataset+'_knn'+str(knn)+'_perc'+str(perc)+'_lam'+str(lam)+'_Knoisy'+str(rand_layers),"rb") as fp:
        list_theta = pickle.load(fp)
    
    
    sample3 = len(list_acc) #random layers added
    
    K = len(list_acc[0])-(5+2) #layers 
    
    method_names = []
    for i in range(0,K):
        method_names.append('layer '+str(i+1))
    method_names = method_names + ['arithmetic', 
    'harmonic',
    'maximum',
    'minimum',
    'geometric',
    'binomial',
    'multi']
    
    matrix_acc_mean_ = np.empty([K+5+2,sample3])
    matrix_acc_std_ = np.empty([K+5+2,sample3])
    for rrr in range(0,sample3): #random layers added 

        matrix_acc = list_acc[rrr]

        #mean and std all methods respect to random networks
        matrix_acc_mean = np.mean(matrix_acc,axis=1)
        matrix_acc_std = np.std(matrix_acc,axis=1)

        matrix_acc_mean_[:,rrr] = matrix_acc_mean
        matrix_acc_std_[:,rrr] = matrix_acc_std

    matrix_acc_mean = np.mean(matrix_acc_mean_,axis=1)
    matrix_acc_std = np.mean(matrix_acc_std_,axis=1)

    matrix_acc_mean_df = pd.DataFrame(list(zip(method_names,matrix_acc_mean,matrix_acc_std)))
    matrix_acc_mean_df.columns = ['method', 'acc_mean','acc_std']
    #matrix_acc_mean_df = matrix_acc_mean_df.sort_values(by=['acc_mean'], ascending=False)
    
    C = dict_dataset_C[dataset]
    matrix_theta_mean_ = np.empty([sample3,2,K+1,C])
    matrix_theta_std_ = np.empty([sample3,2,K+1,C])
    for rrr in range(0,sample3): #random layers added

        matrix_theta = list_theta[rrr]

        #mean and std all methods respect to random networks
        matrix_theta_mean = np.mean(matrix_theta,axis=0)
        matrix_theta_std = np.std(matrix_theta,axis=0)

        matrix_theta_mean_[rrr,:,:,:] = matrix_theta_mean
        matrix_theta_std_[rrr,:,:,:] = matrix_theta_std

    matrix_theta_mean = np.mean(matrix_theta_mean_,axis=0)
    matrix_theta_std = np.mean(matrix_theta_std_,axis=0)

    df1 = pd.DataFrame(list(matrix_theta_mean[0]))
    df1.insert(0, 'method/community', method_names[-2])
    df2 = pd.DataFrame(list(matrix_theta_mean[1]))
    df2.insert(0, 'method/community', method_names[-1])

    matrix_theta_mean_df = pd.concat([df1, df2], axis = 0)
    
    
    return matrix_acc_mean_df,matrix_theta_mean_df

### Tests

#### synthetic_datasets

In [8]:
N = 1200 #number of nodes
C = 3 #number of communities
K = 3 #number layers
perc = 20
lam = 1 #lambda #parameter lower level pbl 

In [ ]:
case = 'info2' 
knn = 5 
for cluster_std in [5,6,7,8]:
    A_save,labels_save,Y_save,matrix_acc,matrix_theta = synthetic_datasets(N,C,K,case,cluster_std,knn,perc,lam)

In [ ]:
case = 'compl' 
knn = 5 
for cluster_std in [2,3,4,5]:
    A_save,labels_save,Y_save,matrix_acc,matrix_theta = synthetic_datasets(N,C,K,case,cluster_std,knn,perc,lam)

In [ ]:
case = 'noisy' #info2,noisy,compl,compl2
knn = 5 
for cluster_std in [2,3,4,5]:
    A_save,labels_save,Y_save,matrix_acc,matrix_theta = synthetic_datasets(N,C,K,case,cluster_std,knn,perc,lam)

In [ ]:
#print results

In [ ]:
case = 'info2' 
print("case "+case)
knn = 5 
for cluster_std in [5,6,7,8]:
    print("std "+str(cluster_std))
    matrix_acc_mean,matrix_theta_mean = results_statistics_synthetic(N,C,K,case,cluster_std,knn,perc,lam)
    display(matrix_acc_mean)
    display(matrix_theta_mean)

In [ ]:
case = 'noisy' #info2,noisy,compl,compl2
print("case "+case)
knn = 5 
for cluster_std in [2,3,4,5]:
    print("std "+str(cluster_std))
    matrix_acc_mean,matrix_theta_mean = results_statistics_synthetic(N,C,K,case,cluster_std,knn,perc,lam)
    display(matrix_acc_mean)
    display(matrix_theta_mean)

In [ ]:
case = 'compl' 
print("case "+case)
knn = 5 
for cluster_std in [2,3,4,5]:
    print("std "+str(cluster_std))
    matrix_acc_mean,matrix_theta_mean = results_statistics_synthetic(N,C,K,case,cluster_std,knn,perc,lam)
    display(matrix_acc_mean)
    display(matrix_theta_mean)

#### real_datasets

In [ ]:
knn = 5
lam = 1 #lambda #parameter lower level pbl
for dataset in datasets_list:
    for perc in [15]: #[1,5,10,15,20,25]
        A,labels,Y_save,matrix_acc,matrix_theta  = real_datasets(dataset,knn,perc,lam)

In [ ]:
#print results
knn = 5
lam = 1 
perc = 15
for dataset in datasets_list:
        print(dataset)
        matrix_acc_mean_df,matrix_theta_mean_df = results_statistics_real(dataset,knn,perc,lam)
        display(matrix_acc_mean_df)
        display(matrix_theta_mean_df)

In [ ]:
#noisy

In [ ]:
knn = 5
lam = 1 #lambda #parameter lower level pbl
perc = 15
for dataset in datasets_list:
    for rand_layers in [1,2]: 
        A_save,labels,Y_save,list_acc,list_theta  = real_datasets_noisy(dataset,knn,perc,lam,rand_layers)

In [ ]:
knn = 5
lam = 1 #lambda #parameter lower level pbl
perc = 15
for dataset in datasets_list:
    for rand_layers in [1,2]: 
        print(dataset+" - "+str(rand_layers)+" random layers")
        matrix_acc_mean_df,matrix_theta_mean_df = results_statistics_real_noisy(dataset,knn,perc,lam,rand_layers)
        display(matrix_acc_mean_df)
        display(matrix_theta_mean_df)